In [37]:
import time
import sys
import numpy as np
sys.path.append('/home/chaofan/powerknowledge/data')
from read_PLAID_data import read_processed_data,get_feature_name
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

start_reading_time = time.time()
feature_select = get_feature_name(
    '/home/chaofan/powerknonawledge/data/source/submetered_zengj/total')
x_train, y_train, index_train = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/training')

x_validation, y_validation, index_validation = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/validation')

x_trainval = np.concatenate((x_train, x_validation), axis=0)
y_trainval = np.concatenate((y_train, y_validation), axis=0)

x_test, y_test, index_test = read_processed_data(
    'type',
    type_header='appliance',
    direaction=1,
    offset=0,
    each_lenth=1,
    feature_select=feature_select,
    source='submetered_zengj/testing')

for ne in [80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300]:
    for md in [6,7,8,9,10,11,12]:
        rf_total = RandomForestClassifier(n_estimators=ne,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        max_depth=md,
                                        max_features='sqrt',
                                        random_state=10)
        rf_total.fit(x_trainval, y_trainval)
        y_test_pred = rf_total.predict(x_test)
        print("ne:%d md:%d Accuracy : %.4g" % (ne,md,metrics.accuracy_score(y_test, y_test_pred)))

print('finished loading data, cost %.3fs' % (time.time() - start_reading_time))

ne:80 md:6 Accuracy : 0.7943
ne:80 md:7 Accuracy : 0.8156
ne:80 md:8 Accuracy : 0.8174
ne:80 md:9 Accuracy : 0.8067
ne:80 md:10 Accuracy : 0.8121
ne:80 md:11 Accuracy : 0.8174
ne:80 md:12 Accuracy : 0.8121
ne:90 md:6 Accuracy : 0.7979
ne:90 md:7 Accuracy : 0.8103
ne:90 md:8 Accuracy : 0.8174
ne:90 md:9 Accuracy : 0.8156
ne:90 md:10 Accuracy : 0.8103
ne:90 md:11 Accuracy : 0.8121
ne:90 md:12 Accuracy : 0.8121
ne:100 md:6 Accuracy : 0.8032
ne:100 md:7 Accuracy : 0.8067
ne:100 md:8 Accuracy : 0.8103
ne:100 md:9 Accuracy : 0.8174
ne:100 md:10 Accuracy : 0.8085
ne:100 md:11 Accuracy : 0.8103
ne:100 md:12 Accuracy : 0.8138
ne:110 md:6 Accuracy : 0.8014
ne:110 md:7 Accuracy : 0.8085
ne:110 md:8 Accuracy : 0.8121
ne:110 md:9 Accuracy : 0.8174
ne:110 md:10 Accuracy : 0.805
ne:110 md:11 Accuracy : 0.8156
ne:110 md:12 Accuracy : 0.8138
ne:120 md:6 Accuracy : 0.8014
ne:120 md:7 Accuracy : 0.8085
ne:120 md:8 Accuracy : 0.8174
ne:120 md:9 Accuracy : 0.8209
ne:120 md:10 Accuracy : 0.8121
ne:120 md:11

In [80]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
i_pp_rms=3.2
P1=150
P0=10
cross_rate=1

x_electric_trainval = []
y_electric_trainval = []
x_low_trainval = []
y_low_trainval = []
x_high_trainval = []
y_high_trainval = []

for i, data in enumerate(x_trainval):
    if data[4] > i_pp_rms*(1/cross_rate):
        x_electric_trainval.append(data)
        y_electric_trainval.append(y_trainval[i])
        continue
    if data[4] >= i_pp_rms*cross_rate:
        x_electric_trainval.append(data)
        y_electric_trainval.append(y_trainval[i])
    if data[7] < P1*cross_rate and data[7]>P0:
        x_low_trainval.append(data)
        y_low_trainval.append(y_trainval[i])
        continue
    if data[7] > P1*(1/cross_rate):
        x_high_trainval.append(data)
        y_high_trainval.append(y_trainval[i])
        continue
    if data[7] >= P1*cross_rate and data[7] <= P1*(1/cross_rate):
        x_low_trainval.append(data)
        y_low_trainval.append(y_trainval[i])
        x_high_trainval.append(data)
        y_high_trainval.append(y_trainval[i])

x_electric_trainval = np.array(x_electric_trainval)
y_electric_trainval = np.array(y_electric_trainval)
x_low_trainval = np.array(x_low_trainval)
y_low_trainval = np.array(y_low_trainval)
x_high_trainval = np.array(x_high_trainval)
y_high_trainval = np.array(y_high_trainval)

x_electric_test = []
y_electric_test = []
x_low_test = []
y_low_test = []
x_high_test = []
y_high_test = []

for i, data in enumerate(x_test):
    if data[4] > i_pp_rms:
        x_electric_test.append(data)
        y_electric_test.append(y_test[i])
        continue
    elif data[7] < P1 and data[7]>P0:
        x_low_test.append(data)
        y_low_test.append(y_test[i])
        continue
    elif data[7] >= P1:
        x_high_test.append(data)
        y_high_test.append(y_test[i])
        continue
# for ne in [30,40,50,60,70,80,90,100,120,140,160]:
#     for md in [6,7,8,9,10,11,12]:
#         rf_electric = RandomForestClassifier(n_estimators=ne,
#                                             min_samples_split=10,
#                                             min_samples_leaf=5,
#                                             max_depth=md,
#                                             max_features='sqrt',
#                                             random_state=30)
#         rf_electric.fit(x_electric_trainval, y_electric_trainval)
#         print("n:%d md:%d Accuracy : %.4g" % (ne,md,metrics.accuracy_score(y_electric_test, rf_electric.predict(x_electric_test))))


rf_electric = RandomForestClassifier(n_estimators=40,
                                    min_samples_split=10,
                                    min_samples_leaf=5,
                                    max_depth=6,
                                    max_features='sqrt',
                                    random_state=30)
rf_electric.fit(x_electric_trainval, y_electric_trainval)
print("Accuracy : %.4g" % metrics.accuracy_score(y_electric_test, rf_electric.predict(x_electric_test)))

# for n_estimators in [80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300]:
#     for md in [6,7,8,9,10,11,12]:
#         rf_low = RandomForestClassifier(n_estimators=n_estimators,
#                                         min_samples_split=10,
#                                         min_samples_leaf=5,
#                                         max_depth=md,
#                                         max_features='sqrt',
#                                         random_state=10)
#         rf_low.fit(x_low_trainval, y_low_trainval)
#         print("n:%d md:%d Accuracy : %.4g" % (n_estimators,md,metrics.accuracy_score(y_low_test, rf_low.predict(x_low_test))))

rf_low = RandomForestClassifier(n_estimators=170,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=11,
                                max_features='sqrt',
                                random_state=30)
rf_low.fit(x_low_trainval, y_low_trainval)
print("Accuracy : %.4g" % (metrics.accuracy_score(y_low_test, rf_low.predict(x_low_test))))

# for ne in [50,60,70,80,90,100,120,140,160,180,200,210,220,230,240,250,260,270,280,290,300]:
#     for md in [6,7,8,9,10,11,12]:
#         rf_high = RandomForestClassifier(n_estimators=ne,
#                                         min_samples_split=10,
#                                         min_samples_leaf=5,
#                                         max_depth=md,
#                                         max_features='sqrt',
#                                         random_state=10)
#         rf_high.fit(x_high_trainval, y_high_trainval)
#         print("n:%d md:%d Accuracy : %.4g" % (ne,md,metrics.accuracy_score(y_high_test, rf_high.predict(x_high_test))))

rf_high = RandomForestClassifier(n_estimators=300,
                                min_samples_split=10,
                                min_samples_leaf=5,
                                max_depth=8,
                                max_features='sqrt',
                                random_state=30)
rf_high.fit(x_high_trainval, y_high_trainval)
print("Accuracy : %.4g" % (metrics.accuracy_score(y_high_test, rf_high.predict(x_high_test))))

print("finish!")

Accuracy : 0.9593
Accuracy : 0.7537
Accuracy : 0.942
finish!


In [58]:
from sklearn.ensemble import RandomForestClassifier
# i_pp_rms=4.2
# P1=100
P0=10
cross_rate=0.95
best_score=0.0
for i_pp_rms in [3.0,3.2,3.4,3.6,3.9,4.0,4.1,4.2,4.3]:
    for P1 in [50,60,70,80,90,100,110,120,130,140,150,170,190,200,300]:
        x_electric_trainval = []
        y_electric_trainval = []
        x_low_trainval = []
        y_low_trainval = []
        x_high_trainval = []
        y_high_trainval = []

        for i, data in enumerate(x_trainval):
            if data[4] > i_pp_rms*(1/cross_rate):
                x_electric_trainval.append(data)
                y_electric_trainval.append(y_trainval[i])
                continue
            elif data[4] >= i_pp_rms: # 
                x_electric_trainval.append(data)
                y_electric_trainval.append(y_trainval[i])
            elif data[7] < P1*cross_rate and data[7]>P0:
                x_low_trainval.append(data)
                y_low_trainval.append(y_trainval[i])
                continue
            elif data[7] > P1*(1/cross_rate):
                x_high_trainval.append(data)
                y_high_trainval.append(y_trainval[i])
                continue
            elif data[7] >= P1 and data[7] <= P1*(1/cross_rate):
                x_high_trainval.append(data)
                y_high_trainval.append(y_trainval[i])
            elif data[7] >= P1*cross_rate and data[7] <= P1*(1/cross_rate):
                x_low_trainval.append(data)
                y_low_trainval.append(y_trainval[i])

        x_electric_trainval = np.array(x_electric_trainval)
        y_electric_trainval = np.array(y_electric_trainval)
        x_low_trainval = np.array(x_low_trainval)
        y_low_trainval = np.array(y_low_trainval)
        x_high_trainval = np.array(x_high_trainval)
        y_high_trainval = np.array(y_high_trainval)

        x_electric_test = []
        y_electric_test = []
        x_low_test = []
        y_low_test = []
        x_high_test = []
        y_high_test = []

        for i, data in enumerate(x_test):
            if data[4] > i_pp_rms:
                x_electric_test.append(data)
                y_electric_test.append(y_test[i])
                continue
            elif data[7] < P1 and data[7]>P0:
                x_low_test.append(data)
                y_low_test.append(y_test[i])
                continue
            elif data[7] >= P1:
                x_high_test.append(data)
                y_high_test.append(y_test[i])
                continue

        x_electric_test = np.array(x_electric_test)
        y_electric_test = np.array(y_electric_test)
        x_low_test = np.array(x_low_test)
        y_low_test = np.array(y_low_test)
        x_high_test = np.array(x_high_test)
        y_high_test = np.array(y_high_test)

        rf_electric = RandomForestClassifier(n_estimators=40,
                                            min_samples_split=10,
                                            min_samples_leaf=5,
                                            max_depth=6,
                                            max_features='sqrt',
                                            random_state=30)
        rf_electric.fit(x_electric_trainval, y_electric_trainval)
        electric_score = metrics.accuracy_score(y_electric_test, rf_electric.predict(x_electric_test))

        rf_low = RandomForestClassifier(n_estimators=170,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        max_depth=11,
                                        max_features='sqrt',
                                        random_state=30)
        rf_low.fit(x_low_trainval, y_low_trainval)
        low_score = metrics.accuracy_score(y_low_test, rf_low.predict(x_low_test))

        rf_high = RandomForestClassifier(n_estimators=300,
                                        min_samples_split=10,
                                        min_samples_leaf=5,
                                        max_depth=8,
                                        max_features='sqrt',
                                        random_state=30)
        rf_high.fit(x_high_trainval, y_high_trainval)
        high_score = metrics.accuracy_score(y_high_test, rf_high.predict(x_high_test))
        
        electric_len = len(x_electric_test)
        low_len = len(x_low_test)
        high_len = len(x_high_test)
        total_len = electric_len + low_len + high_len
        electric_rate = electric_len/total_len
        low_rate = low_len/total_len
        high_rate = high_len/total_len
        ensemble_accuracy=electric_rate*electric_score+low_rate*low_score+high_rate*high_score
        print("i_pp_rms:%f  P1:%f  acc:%f" %(i_pp_rms,P1,ensemble_accuracy))



i_pp_rms:3.000000  P1:50.000000  acc:0.793594
i_pp_rms:3.000000  P1:60.000000  acc:0.802491
i_pp_rms:3.000000  P1:70.000000  acc:0.823843
i_pp_rms:3.000000  P1:80.000000  acc:0.832740
i_pp_rms:3.000000  P1:90.000000  acc:0.829181
i_pp_rms:3.000000  P1:100.000000  acc:0.832740
i_pp_rms:3.000000  P1:110.000000  acc:0.834520
i_pp_rms:3.000000  P1:120.000000  acc:0.809609
i_pp_rms:3.000000  P1:130.000000  acc:0.816726
i_pp_rms:3.000000  P1:140.000000  acc:0.830961
i_pp_rms:3.000000  P1:150.000000  acc:0.829181
i_pp_rms:3.000000  P1:170.000000  acc:0.827402
i_pp_rms:3.000000  P1:190.000000  acc:0.807829
i_pp_rms:3.000000  P1:200.000000  acc:0.836299
i_pp_rms:3.000000  P1:300.000000  acc:0.832740
i_pp_rms:3.200000  P1:50.000000  acc:0.822064
i_pp_rms:3.200000  P1:60.000000  acc:0.836299
i_pp_rms:3.200000  P1:70.000000  acc:0.823843
i_pp_rms:3.200000  P1:80.000000  acc:0.839858
i_pp_rms:3.200000  P1:90.000000  acc:0.829181
i_pp_rms:3.200000  P1:100.000000  acc:0.846975
i_pp_rms:3.200000  P1:1

In [42]:
from sklearn.ensemble import RandomForestClassifier
P0=10
cross_rate=1
best_score=0.0
best_i_pp_rms=0
best_P1=0
best_P2=0

for i_pp_rms in [3.0,3.2,3.4,3.6,3.9,4.0,4.1,4.2,4.3]:
    for P1 in [50,60,70,80,90,100,110,120,130,140,150,170,190,200,300]:
        for P2 in [30,40,50,60,70,80,90,100,150,200,300]:
            x_electric_trainval = []
            y_electric_trainval = []
            x_low_trainval = []
            y_low_trainval = []
            x_medium_trainval = []
            y_medium_trainval = []
            x_high_trainval = []
            y_high_trainval = []

            for i, data in enumerate(x_trainval):
                if data[4] > i_pp_rms*(1/cross_rate):   # 非重叠
                    x_electric_trainval.append(data)
                    y_electric_trainval.append(y_trainval[i])
                    continue
                elif data[4] >= i_pp_rms*cross_rate:    # 重叠
                    x_electric_trainval.append(data)
                    y_electric_trainval.append(y_trainval[i])
                elif data[7] > P0 and data[7] < P1*cross_rate:    # 非重叠
                    x_low_trainval.append(data)
                    y_low_trainval.append(y_trainval[i])
                    continue
                elif data[7] >= P1*cross_rate and data[7] <= P1*(1/cross_rate): # 重叠
                    x_low_trainval.append(data)
                    y_low_trainval.append(y_trainval[i])
                    x_medium_trainval.append(data)
                    y_medium_trainval.append(y_trainval[i])
                elif data[7] > P1*(1/cross_rate) and data[7] < (P1+P2)*cross_rate:   # 非重叠
                    x_medium_trainval.append(data)
                    y_medium_trainval.append(y_trainval[i])
                    continue
                elif data[7] >= (P1+P2)*cross_rate and data[7] <= (P1+P2)*(1/cross_rate):   # 重叠
                    x_medium_trainval.append(data)
                    y_medium_trainval.append(y_trainval[i])
                    x_high_trainval.append(data)
                    y_high_trainval.append(y_trainval[i])
                elif data[7] > (P1+P2)*(1/cross_rate):  # 非重叠
                    x_high_trainval.append(data)
                    y_high_trainval.append(y_trainval[i])

            x_electric_trainval = np.array(x_electric_trainval)
            y_electric_trainval = np.array(y_electric_trainval)
            x_low_trainval = np.array(x_low_trainval)
            y_low_trainval = np.array(y_low_trainval)
            x_medium_trainval = np.array(x_medium_trainval)
            y_medium_trainval = np.array(y_medium_trainval)
            x_high_trainval = np.array(x_high_trainval)
            y_high_trainval = np.array(y_high_trainval)
            if len(x_medium_trainval)==0 or len(x_low_trainval)==0 or len(x_high_trainval)==0:
                continue

            x_electric_test = []
            y_electric_test = []
            x_low_test = []
            y_low_test = []
            x_medium_test = []
            y_medium_test = []
            x_high_test = []
            y_high_test = []

            for i, data in enumerate(x_test):
                if data[4] > i_pp_rms:
                    x_electric_test.append(data)
                    y_electric_test.append(y_test[i])
                    continue
                elif data[7] > P0 and data[7] <= P1:
                    x_low_test.append(data)
                    y_low_test.append(y_test[i])
                    continue
                elif data[7] > P1 and data[7] <= (P1+P2):
                    x_medium_test.append(data)
                    y_medium_test.append(y_test[i])
                    continue
                elif data[7] > (P1+P2):
                    x_high_test.append(data)
                    y_high_test.append(y_test[i])
                    continue

            x_electric_test = np.array(x_electric_test)
            y_electric_test = np.array(y_electric_test)
            x_low_test = np.array(x_low_test)
            y_low_test = np.array(y_low_test)
            x_medium_test = np.array(x_medium_test)
            y_medium_test = np.array(y_medium_test)
            x_high_test = np.array(x_high_test)
            y_high_test = np.array(y_high_test)
            if len(x_medium_test)==0 or len(x_low_test)==0 or len(x_high_test)==0:
                continue

            rf_electric = RandomForestClassifier(n_estimators=40,
                                                min_samples_split=10,
                                                min_samples_leaf=5,
                                                max_depth=6,
                                                max_features='sqrt',
                                                random_state=30)
            rf_electric.fit(x_electric_trainval, y_electric_trainval)
            electric_score = metrics.accuracy_score(y_electric_test, rf_electric.predict(x_electric_test))

            rf_low = RandomForestClassifier(n_estimators=170,
                                            min_samples_split=10,
                                            min_samples_leaf=5,
                                            max_depth=11,
                                            max_features='sqrt',
                                            random_state=30)
            rf_low.fit(x_low_trainval, y_low_trainval)
            low_score = metrics.accuracy_score(y_low_test, rf_low.predict(x_low_test))

            rf_medium = RandomForestClassifier(n_estimators=300,
                                            min_samples_split=10,
                                            min_samples_leaf=5,
                                            max_depth=11,
                                            max_features='sqrt',
                                            random_state=30)
            rf_medium.fit(x_medium_trainval, y_medium_trainval)
            medium_score = metrics.accuracy_score(y_medium_test, rf_medium.predict(x_medium_test))

            rf_high = RandomForestClassifier(n_estimators=300,
                                            min_samples_split=10,
                                            min_samples_leaf=5,
                                            max_depth=8,
                                            max_features='sqrt',
                                            random_state=30)
            rf_high.fit(x_high_trainval, y_high_trainval)
            high_score = metrics.accuracy_score(y_high_test, rf_high.predict(x_high_test))
            
            electric_len = len(x_electric_test)
            low_len = len(x_low_test)
            medium_len = len(x_medium_test)
            high_len = len(x_high_test)
            total_len = electric_len + low_len + medium_len + high_len
            electric_rate = electric_len/total_len
            low_rate = low_len/total_len
            medium_rate = medium_len/total_len
            high_rate = high_len/total_len
            ensemble_accuracy = electric_rate*electric_score + low_rate*low_score + medium_rate*medium_score + high_rate*high_score
            if ensemble_accuracy>best_score:
                best_score=ensemble_accuracy
                best_i_pp_rms=i_pp_rms
                best_P1=P1
                best_P2=P2
            print("i_pp_rms:%.2f  P1:%d P2:%d acc:%f" %(i_pp_rms,P1,P2,ensemble_accuracy))



 acc:0.818018
i_pp_rms:3.90  P1:80 P2:70 acc:0.816216
i_pp_rms:3.90  P1:80 P2:80 acc:0.819820
i_pp_rms:3.90  P1:80 P2:90 acc:0.819820
i_pp_rms:3.90  P1:80 P2:100 acc:0.819820
i_pp_rms:3.90  P1:80 P2:150 acc:0.818018
i_pp_rms:3.90  P1:80 P2:200 acc:0.821622
i_pp_rms:3.90  P1:80 P2:300 acc:0.812613
i_pp_rms:3.90  P1:90 P2:30 acc:0.828829
i_pp_rms:3.90  P1:90 P2:40 acc:0.821622
i_pp_rms:3.90  P1:90 P2:50 acc:0.814414
i_pp_rms:3.90  P1:90 P2:60 acc:0.812613
i_pp_rms:3.90  P1:90 P2:70 acc:0.816216
i_pp_rms:3.90  P1:90 P2:80 acc:0.816216
i_pp_rms:3.90  P1:90 P2:90 acc:0.816216
i_pp_rms:3.90  P1:90 P2:100 acc:0.827027
i_pp_rms:3.90  P1:90 P2:150 acc:0.818018
i_pp_rms:3.90  P1:90 P2:200 acc:0.821622
i_pp_rms:3.90  P1:90 P2:300 acc:0.796396
i_pp_rms:3.90  P1:100 P2:30 acc:0.839640
i_pp_rms:3.90  P1:100 P2:40 acc:0.832432
i_pp_rms:3.90  P1:100 P2:50 acc:0.830631
i_pp_rms:3.90  P1:100 P2:60 acc:0.834234
i_pp_rms:3.90  P1:100 P2:70 acc:0.834234
i_pp_rms:3.90  P1:100 P2:80 acc:0.834234
i_pp_rms:3.9

In [92]:
def total_clf(x_test,prob_threshold=0.6,rate=0.9):
    test_pred=[]
    for i,data in enumerate(x_test):
        data=data.reshape(1,-1)
        data_prob_list=[]
        data_pred_list=[]
        data_prob_list.append(np.max(rf_electric.predict_proba(data)))
        data_pred_list.append(rf_electric.predict(data))
        data_prob_list.append(np.max(rf_low.predict_proba(data)))
        data_pred_list.append(rf_low.predict(data))
        data_prob_list.append(np.max(rf_high.predict_proba(data)))
        data_pred_list.append(rf_high.predict(data))
        if data[0,4]>3.2:
            if data_prob_list[0]>prob_threshold:
                test_pred.append(data_pred_list[0])
                continue
            if data[0,7]<150:
                if data_prob_list[1]>prob_threshold*rate:
                    test_pred.append(data_pred_list[1])
                    continue
                elif data_prob_list[2]>prob_threshold*rate*rate:
                    test_pred.append(data_pred_list[2])
                    continue
            else:
                if data_prob_list[2]>prob_threshold*rate*rate:
                    test_pred.append(data_pred_list[2])
                    continue
                elif data_prob_list[1]>prob_threshold*rate:
                    test_pred.append(data_pred_list[1])
                    continue
            test_pred.append(data_pred_list[np.argmax(data_prob_list)])
            continue
        elif data[0,7]<150:
            if data_prob_list[1]>prob_threshold:
                test_pred.append(data_pred_list[1])
                continue
            if data_prob_list[2]>prob_threshold*rate:
                test_pred.append(data_pred_list[2])
                continue
            if data_prob_list[0]>prob_threshold*rate*rate:
                test_pred.append(data_pred_list[0])
                continue
            test_pred.append(data_pred_list[np.argmax(data_prob_list)])
            continue
        elif data[0,7]>=150:
            if data_prob_list[2]>prob_threshold:
                test_pred.append(data_pred_list[2])
                continue
            if data_prob_list[1]>prob_threshold*rate:
                test_pred.append(data_pred_list[1])
                continue
            if data_prob_list[0]>prob_threshold*rate*rate:
                test_pred.append(data_pred_list[0])
                continue
            test_pred.append(data_pred_list[np.argmax(data_prob_list)])
            continue
    return np.array(test_pred)

def calc_acc(labels,predictions):
    counts=0
    if len(labels)!=len(predictions):
        return 0
    else:
        for i,pred in enumerate(predictions):
            if pred==labels[i]:
                counts+=1
        return counts/len(predictions)
x_460=x_test[460,:].reshape(1,-1)
preds=np.array(total_clf(x_460,prob_threshold=threshold,rate=rate))

for threshold in [0,0.3,0.35,0.4,0.5,0.6]:
    for rate in [0.7,0.8,0.9,1]:
        preds=np.array(total_clf(x_test,prob_threshold=threshold,rate=rate))
        preds=preds.reshape(len(preds),)
        print("thre:%f,rate:%f,calc:%.3f" %(threshold,rate,calc_acc(y_test,preds)))


thre:0.000000,rate:0.700000,calc:0.878
thre:0.000000,rate:0.800000,calc:0.878
thre:0.000000,rate:0.900000,calc:0.878
thre:0.000000,rate:1.000000,calc:0.878
thre:0.300000,rate:0.700000,calc:0.865
thre:0.300000,rate:0.800000,calc:0.865
thre:0.300000,rate:0.900000,calc:0.869
thre:0.300000,rate:1.000000,calc:0.869
thre:0.350000,rate:0.700000,calc:0.849
thre:0.350000,rate:0.800000,calc:0.853
thre:0.350000,rate:0.900000,calc:0.853
thre:0.350000,rate:1.000000,calc:0.853
thre:0.400000,rate:0.700000,calc:0.833
thre:0.400000,rate:0.800000,calc:0.824
thre:0.400000,rate:0.900000,calc:0.823
thre:0.400000,rate:1.000000,calc:0.826
thre:0.500000,rate:0.700000,calc:0.754
thre:0.500000,rate:0.800000,calc:0.761
thre:0.500000,rate:0.900000,calc:0.778
thre:0.500000,rate:1.000000,calc:0.780
thre:0.600000,rate:0.700000,calc:0.676
thre:0.600000,rate:0.800000,calc:0.748
thre:0.600000,rate:0.900000,calc:0.771
thre:0.600000,rate:1.000000,calc:0.771
